## Data prep

In [63]:
from census_utils import *
import pandas as pd
import geopandas as gpd
from random import random
from math import sqrt
from scipy.spatial import KDTree
import numpy as np

In [2]:
def load_data():
    return pd.read_csv(get_synthetic_out_file())

In [3]:
df = load_data()

In [4]:
df.head()

,YEAR,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,TRACTA,BLKGRPA,BLOCKA,NAME,...,AS,H_PI,OTH,TWO_OR_MORE,NUM_HISP,18_PLUS,HH_NUM,ACCURACY,AGE_ACCURACY,identifier
0,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,0,1,0,1,True,19501001005
1,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,0,1,1,1,True,19501001005
2,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,0,1,2,1,True,19501001005
3,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,0,1,3,1,True,19501001005
4,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,0,1,4,1,True,19501001005


In [5]:
num_rows = len(df)

In [6]:
print(num_rows)

3612858


In [7]:
dist_u = {
    4: .05,
    3: .15,
    2: .3,
    1: .5
}

In [8]:
dist_n = {k: v*num_rows for k, v in dist_u.items()}

In [9]:
dist_n

{4: 180642.90000000002, 3: 541928.7, 2: 1083857.4, 1: 1806429.0}

In [10]:
flagging = ['W', 'B', 'AI_AN', 'AS', 'H_PI', 'OTH', 'TWO_OR_MORE', 'NUM_HISP']

In [11]:
counts = df[flagging].groupby(flagging).size().reset_index()

In [12]:
merged = df.merge(counts,
         how='left',
         on=flagging,
         validate='many_to_one',
).rename({0: 'frequency'}, axis=1)
merged.head()

,YEAR,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,TRACTA,BLKGRPA,BLOCKA,NAME,...,H_PI,OTH,TWO_OR_MORE,NUM_HISP,18_PLUS,HH_NUM,ACCURACY,AGE_ACCURACY,identifier,frequency
0,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,1,0,1,True,19501001005,552198
1,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,1,1,1,True,19501001005,552198
2,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,1,2,1,True,19501001005,552198
3,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,1,3,1,True,19501001005,552198
4,2010,Georgia,13,Appling County,1,90078,950100,1,1005,Block 1005,...,0,0,0,0,1,4,1,True,19501001005,827171


In [13]:
merged.sort_values(by=['BLOCK_TOTAL', 'frequency'], axis=0, inplace=True)

In [14]:
vec_n = [[i] * int(dist_n[i]) for i in (4, 3, 2, 1)]

In [15]:
l = []
for v in vec_n:
    l += v
if len(l) < num_rows:
    l += [1] * (num_rows - len(l))
print(l[:5])
print(len(l))

[4, 4, 4, 4, 4]
3612858


In [16]:
merged['U'] = l

In [17]:
merged.head()

,YEAR,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,TRACTA,BLKGRPA,BLOCKA,NAME,...,OTH,TWO_OR_MORE,NUM_HISP,18_PLUS,HH_NUM,ACCURACY,AGE_ACCURACY,identifier,frequency,U
2636079,2010,Georgia,13,Houston County,153,92352,21400,1,1034,Block 1034,...,0,0,1,1,0,1,True,153214001034,102,4
3511182,2010,Georgia,13,Ware County,299,93240,950200,3,3025,Block 3025,...,0,0,1,1,0,1,True,2999502003025,102,4
175554,2010,Georgia,13,Bleckley County,23,90708,790100,1,1102,Block 1102,...,0,0,0,1,0,1,True,237901001102,268,4
240473,2010,Georgia,13,Butts County,35,91182,150100,4,4082,Block 4082,...,0,0,0,1,0,1,True,351501004082,268,4
607044,2010,Georgia,13,Clayton County,63,91200,40412,3,3031,Block 3031,...,0,0,0,1,0,1,True,63404123031,268,4


In [18]:
merged[['BLOCK_TOTAL', 'frequency', 'U']].head()

,BLOCK_TOTAL,frequency,U
2636079,1,102,4
3511182,1,102,4
175554,1,268,4
240473,1,268,4
607044,1,268,4


In [19]:
#merged.loc[merged['BLOCK_TOTAL'] <= 9, 'U'] = 4

In [20]:
#merged[merged['BLOCK_TOTAL'] <= 9].tail()

,YEAR,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,TRACTA,BLKGRPA,BLOCKA,NAME,...,OTH,TWO_OR_MORE,NUM_HISP,18_PLUS,HH_NUM,ACCURACY,AGE_ACCURACY,identifier,frequency,U
3609794,2010,Georgia,13,Worth County,321,92412,950400,4,4055,Block 4055,...,0,0,0,2,2,1,True,3219504004055,827171,4
3609924,2010,Georgia,13,Worth County,321,92412,950400,4,4092,Block 4092,...,0,0,0,2,1,1,True,3219504004092,827171,4
3609969,2010,Georgia,13,Worth County,321,92412,950400,4,4109,Block 4109,...,0,0,0,2,0,1,True,3219504004109,827171,4
3610084,2010,Georgia,13,Worth County,321,92412,950400,4,4164,Block 4164,...,0,0,0,2,1,1,True,3219504004164,827171,4
3611062,2010,Georgia,13,Worth County,321,92940,950500,2,2034,Block 2034,...,0,0,0,2,5,1,True,3219505002034,827171,4


In [21]:
merged.head()

,YEAR,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,TRACTA,BLKGRPA,BLOCKA,NAME,...,OTH,TWO_OR_MORE,NUM_HISP,18_PLUS,HH_NUM,ACCURACY,AGE_ACCURACY,identifier,frequency,U
2636079,2010,Georgia,13,Houston County,153,92352,21400,1,1034,Block 1034,...,0,0,1,1,0,1,True,153214001034,102,4
3511182,2010,Georgia,13,Ware County,299,93240,950200,3,3025,Block 3025,...,0,0,1,1,0,1,True,2999502003025,102,4
175554,2010,Georgia,13,Bleckley County,23,90708,790100,1,1102,Block 1102,...,0,0,0,1,0,1,True,237901001102,268,4
240473,2010,Georgia,13,Butts County,35,91182,150100,4,4082,Block 4082,...,0,0,0,1,0,1,True,351501004082,268,4
607044,2010,Georgia,13,Clayton County,63,91200,40412,3,3031,Block 3031,...,0,0,0,1,0,1,True,63404123031,268,4


In [22]:
merged['prob'] = merged['U']

In [23]:
merged['prob'].replace({4: 1, 3: .6, 2: .3, 1: .1}, inplace=True)

In [24]:
del merged['identifier']

In [25]:
ID_COLS = ['TRACTA', 'COUNTYA', 'BLOCKA']
def make_identifier_synth(df):
    id_lens = [6, 3, 4]
    str_cols = [col + '_str' for col in ID_COLS]
    for col, l, col_s in zip(ID_COLS, id_lens, str_cols):
        assert max(num_digits(s) for s in df[col].unique()) <= l
        df[col_s] = df[col].astype(str).str.zfill(l)
    df['id'] = df[str_cols].astype(str).agg('-'.join, axis=1)
    for col_s in str_cols:
        del df[col_s]

In [26]:
make_identifier_synth(merged)

In [27]:
merged.head()

,YEAR,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,TRACTA,BLKGRPA,BLOCKA,NAME,...,TWO_OR_MORE,NUM_HISP,18_PLUS,HH_NUM,ACCURACY,AGE_ACCURACY,frequency,U,prob,id
2636079,2010,Georgia,13,Houston County,153,92352,21400,1,1034,Block 1034,...,0,1,1,0,1,True,102,4,1.0,021400-153-1034
3511182,2010,Georgia,13,Ware County,299,93240,950200,3,3025,Block 3025,...,0,1,1,0,1,True,102,4,1.0,950200-299-3025
175554,2010,Georgia,13,Bleckley County,23,90708,790100,1,1102,Block 1102,...,0,0,1,0,1,True,268,4,1.0,790100-023-1102
240473,2010,Georgia,13,Butts County,35,91182,150100,4,4082,Block 4082,...,0,0,1,0,1,True,268,4,1.0,150100-035-4082
607044,2010,Georgia,13,Clayton County,63,91200,40412,3,3031,Block 3031,...,0,0,1,0,1,True,268,4,1.0,040412-063-3031


In [28]:
merged['hh_str'] = merged['HH_NUM'].astype(str).str.zfill(4)
merged['household.id'] = merged[['id', 'hh_str']].astype(str).agg('-'.join, axis=1)
del merged['hh_str']

In [29]:
merged.head()

,YEAR,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,TRACTA,BLKGRPA,BLOCKA,NAME,...,NUM_HISP,18_PLUS,HH_NUM,ACCURACY,AGE_ACCURACY,frequency,U,prob,id,household.id
2636079,2010,Georgia,13,Houston County,153,92352,21400,1,1034,Block 1034,...,1,1,0,1,True,102,4,1.0,021400-153-1034,021400-153-1034-0000
3511182,2010,Georgia,13,Ware County,299,93240,950200,3,3025,Block 3025,...,1,1,0,1,True,102,4,1.0,950200-299-3025,950200-299-3025-0000
175554,2010,Georgia,13,Bleckley County,23,90708,790100,1,1102,Block 1102,...,0,1,0,1,True,268,4,1.0,790100-023-1102,790100-023-1102-0000
240473,2010,Georgia,13,Butts County,35,91182,150100,4,4082,Block 4082,...,0,1,0,1,True,268,4,1.0,150100-035-4082,150100-035-4082-0000
607044,2010,Georgia,13,Clayton County,63,91200,40412,3,3031,Block 3031,...,0,1,0,1,True,268,4,1.0,040412-063-3031,040412-063-3031-0000


## Load geo data

In [30]:
def load_shape_data(area):
    block_map = gpd.read_file(get_shape_file(area))
    return block_map.to_crs("EPSG:3395")

In [31]:
block_geo = load_shape_data('BLOCK')

In [32]:
block_geo.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,GISJOIN,Shape_area,Shape_len,geometry
0,13,007,960100,2050,130079601002050,Block 2050,G5040,R,None,None,S,612528,0,+31.3277979,-084.4160009,G13000709601002050,6.125277e+05,3715.902259,"POLYGON ((-9396469.535 3653263.291, -9396474.0..."
1,13,007,960100,1149,130079601001149,Block 1149,G5040,R,None,None,S,5854,0,+31.3484357,-084.3074734,G13000709601001149,5.853541e+03,702.869588,"POLYGON ((-9384854.459 3655836.851, -9384949.0..."
2,13,007,960100,1209,130079601001209,Block 1209,G5040,R,None,None,S,3861,0,+31.3689730,-084.2441048,G13000709601001209,3.860879e+03,312.937685,"POLYGON ((-9377961.000 3658476.737, -9377972.6..."
3,13,007,960100,1088,130079601001088,Block 1088,G5040,R,None,None,S,12284613,46705,+31.3869852,-084.3178991,G13000709601001088,1.233132e+07,20396.775829,"POLYGON ((-9385835.963 3656145.176, -9386135.9..."
4,13,007,960100,1163,130079601001163,Block 1163,G5040,R,None,None,S,781861,7655,+31.4169671,-084.2312666,G13000709601001163,7.895162e+05,3865.045979,"POLYGON ((-9375959.921 3665128.695, -9375969.7..."


In [33]:
ID_COLS = ['TRACTCE10', 'COUNTYFP10', 'BLOCKCE10']
def make_identifier_synth_geo(df):
    id_lens = [6, 3, 4]
    str_cols = [col + '_str' for col in ID_COLS]
    for col, l, col_s in zip(ID_COLS, id_lens, str_cols):
        assert max(num_digits(s) for s in df[col].unique()) <= l
        df[col_s] = df[col].astype(str).str.zfill(l)
    df['id'] = df[str_cols].astype(str).agg('-'.join, axis=1)
    for col_s in str_cols:
        del df[col_s]

In [34]:
make_identifier_synth_geo(block_geo)

In [35]:
block_geo.head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,GISJOIN,Shape_area,Shape_len,geometry,id
0,13,007,960100,2050,130079601002050,Block 2050,G5040,R,None,None,S,612528,0,+31.3277979,-084.4160009,G13000709601002050,6.125277e+05,3715.902259,"POLYGON ((-9396469.535 3653263.291, -9396474.0...",960100-007-2050
1,13,007,960100,1149,130079601001149,Block 1149,G5040,R,None,None,S,5854,0,+31.3484357,-084.3074734,G13000709601001149,5.853541e+03,702.869588,"POLYGON ((-9384854.459 3655836.851, -9384949.0...",960100-007-1149
2,13,007,960100,1209,130079601001209,Block 1209,G5040,R,None,None,S,3861,0,+31.3689730,-084.2441048,G13000709601001209,3.860879e+03,312.937685,"POLYGON ((-9377961.000 3658476.737, -9377972.6...",960100-007-1209
3,13,007,960100,1088,130079601001088,Block 1088,G5040,R,None,None,S,12284613,46705,+31.3869852,-084.3178991,G13000709601001088,1.233132e+07,20396.775829,"POLYGON ((-9385835.963 3656145.176, -9386135.9...",960100-007-1088
4,13,007,960100,1163,130079601001163,Block 1163,G5040,R,None,None,S,781861,7655,+31.4169671,-084.2312666,G13000709601001163,7.895162e+05,3865.045979,"POLYGON ((-9375959.921 3665128.695, -9375969.7...",960100-007-1163


In [36]:
merged = merged.merge(
    block_geo[['INTPTLAT10', 'INTPTLON10', 'id']],
    on='id',
    how='left',
    validate='many_to_one',
)

In [37]:
merged['INTPTLAT10'] = pd.to_numeric(merged['INTPTLAT10'])
merged['INTPTLON10'] = pd.to_numeric(merged['INTPTLON10'])
merged.head()

,YEAR,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,TRACTA,BLKGRPA,BLOCKA,NAME,...,HH_NUM,ACCURACY,AGE_ACCURACY,frequency,U,prob,id,household.id,INTPTLAT10,INTPTLON10
0,2010,Georgia,13,Houston County,153,92352,21400,1,1034,Block 1034,...,0,1,True,102,4,1.0,021400-153-1034,021400-153-1034-0000,32.446184,-83.697026
1,2010,Georgia,13,Ware County,299,93240,950200,3,3025,Block 3025,...,0,1,True,102,4,1.0,950200-299-3025,950200-299-3025-0000,31.253674,-82.473074
2,2010,Georgia,13,Bleckley County,23,90708,790100,1,1102,Block 1102,...,0,1,True,268,4,1.0,790100-023-1102,790100-023-1102-0000,32.402761,-83.335840
3,2010,Georgia,13,Butts County,35,91182,150100,4,4082,Block 4082,...,0,1,True,268,4,1.0,150100-035-4082,150100-035-4082-0000,33.197020,-83.833332
4,2010,Georgia,13,Clayton County,63,91200,40412,3,3031,Block 3031,...,0,1,True,268,4,1.0,040412-063-3031,040412-063-3031-0000,33.575850,-84.326003


In [38]:
old_merged = merged.copy()

## Build distance data structures

In [60]:
all_num_age_pairs = set(zip(merged['TOTAL'], merged['18_PLUS']))
print(all_num_age_pairs)

{(12, 4), (4, 0), (5, 1), (14, 13), (10, 6), (9, 8), (11, 5), (2, 2), (13, 8), (6, 2), (7, 1), (4, 2), (5, 3), (8, 2), (9, 1), (11, 7), (15, 7), (6, 4), (16, 6), (7, 3), (14, 8), (5, 5), (8, 4), (9, 3), (11, 9), (13, 3), (15, 0), (6, 6), (7, 5), (18, 5), (3, 1), (9, 5), (11, 2), (13, 5), (15, 2), (7, 7), (12, 6), (3, 3), (5, 0), (17, 11), (9, 7), (11, 4), (10, 8), (13, 7), (15, 4), (6, 1), (12, 8), (5, 2), (4, 4), (14, 14), (9, 9), (10, 1), (10, 10), (13, 9), (15, 6), (7, 2), (12, 10), (14, 7), (5, 4), (9, 2), (8, 6), (10, 3), (1, 0), (13, 2), (11, 11), (16, 7), (7, 4), (12, 3), (3, 0), (12, 12), (14, 9), (8, 8), (10, 5), (13, 4), (2, 1), (16, 9), (12, 5), (3, 2), (4, 1), (8, 1), (10, 7), (11, 6), (6, 3), (20, 2), (12, 7), (14, 4), (4, 3), (8, 3), (10, 9), (11, 8), (15, 8), (6, 5), (12, 9), (14, 6), (8, 5), (10, 2), (9, 4), (11, 1), (11, 10), (13, 13), (7, 6), (12, 2), (12, 11), (8, 7), (10, 4), (1, 1), (9, 6), (2, 0), (11, 3), (13, 6)}


In [102]:
trees = {}
indices = {}
for t, a in all_num_age_pairs:
    matches = merged[(merged['TOTAL'] == t) & (merged['18_PLUS'] == a)]
    pts = np.array([matches['INTPTLAT10'], matches['INTPTLON10']]).T
    print((t, a), pts.shape)
    indices[(t, a)] = {i: index for i, (index, row) in enumerate(matches.iterrows())}
    trees[(t, a)] = KDTree(pts)

(12, 4) (762, 2)
(4, 0) (6, 2)
(5, 1) (4004, 2)
(14, 13) (1, 2)
(10, 6) (4067, 2)
(9, 8) (223, 2)
(11, 5) (314, 2)
(2, 2) (1128891, 2)
(13, 8) (124, 2)
(6, 2) (33634, 2)
(7, 1) (442, 2)
(4, 2) (431545, 2)
(5, 3) (39513, 2)
(8, 2) (1622, 2)
(9, 1) (74, 2)
(11, 7) (296, 2)
(15, 7) (306, 2)
(6, 4) (27627, 2)
(16, 6) (89, 2)
(7, 3) (6613, 2)
(14, 8) (120, 2)
(5, 5) (12515, 2)
(8, 4) (3865, 2)
(9, 3) (2415, 2)
(11, 9) (10, 2)
(13, 3) (10, 2)
(15, 0) (23, 2)
(6, 6) (3056, 2)
(7, 5) (6274, 2)
(18, 5) (564, 2)
(3, 1) (46403, 2)
(9, 5) (1201, 2)
(11, 2) (64, 2)
(13, 5) (20, 2)
(15, 2) (564, 2)
(7, 7) (446, 2)
(12, 6) (826, 2)
(3, 3) (251480, 2)
(5, 0) (2, 2)
(17, 11) (235, 2)
(9, 7) (578, 2)
(11, 4) (312, 2)
(10, 8) (378, 2)
(13, 7) (8, 2)
(15, 4) (1, 2)
(6, 1) (955, 2)
(12, 8) (154, 2)
(5, 2) (159910, 2)
(4, 4) (46380, 2)
(14, 14) (170, 2)
(9, 9) (186, 2)
(10, 1) (4, 2)
(10, 10) (62, 2)
(13, 9) (1383, 2)
(15, 6) (85, 2)
(7, 2) (7983, 2)
(12, 10) (156, 2)
(14, 7) (28, 2)
(5, 4) (19560, 2)
(9, 2

In [103]:
print(indices[(7, 6)])

{0: 57826, 1: 57843, 2: 57924, 3: 58278, 4: 58334, 5: 58531, 6: 58539, 7: 58543, 8: 58556, 9: 58564, 10: 58705, 11: 58706, 12: 58727, 13: 71300, 14: 71537, 15: 85637, 16: 85639, 17: 85815, 18: 85823, 19: 85825, 20: 99649, 21: 99870, 22: 99871, 23: 100038, 24: 113713, 25: 114214, 26: 114305, 27: 114542, 28: 114547, 29: 114582, 30: 114587, 31: 114738, 32: 114740, 33: 114741, 34: 114747, 35: 114749, 36: 114757, 37: 114758, 38: 129869, 39: 130040, 40: 130299, 41: 130309, 42: 130333, 43: 130465, 44: 145307, 45: 145868, 46: 145869, 47: 145870, 48: 146173, 49: 146174, 50: 146184, 51: 146338, 52: 146347, 53: 146359, 54: 146371, 55: 161751, 56: 162406, 57: 162407, 58: 162805, 59: 162992, 60: 162995, 61: 179399, 62: 179593, 63: 179594, 64: 179707, 65: 180008, 66: 180020, 67: 180021, 68: 180022, 69: 180031, 70: 180036, 71: 180224, 72: 180235, 73: 180245, 74: 180248, 75: 180267, 76: 196405, 77: 197079, 78: 197455, 79: 197459, 80: 197480, 81: 197493, 82: 197496, 83: 197500, 84: 197505, 85: 197719, 

## Swapping

In [157]:
def block_distance(row):
    lat1 = row['INTPTLAT10']
    lat2 = row['other_lat']
    lon1 = row['INTPTLON10']
    lon2 = row['other_lon']
    return sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

def find_k_closest(row, df, k):
    t = row['TOTAL']
    a = row['18_PLUS']
    tree = trees[(t, a)]
    inds = indices[(t, a)]
    if len(inds) <= k:
        return df[(df['TOTAL'] == t) & (df['18_PLUS'] == a)]
    lat = row['INTPTLAT10']
    lon = row['INTPTLON10']
    num_to_query = k+1
    while num_to_query <= len(inds):
        dists, candidates = tree.query((lat, lon), num_to_query)
#         print(candidates)
        first_non_zero = 0
        while dists[first_non_zero] == 0:
            first_non_zero += 1
        dists = dists[first_non_zero:]
        candidates = candidates[first_non_zero:]
#         print(dists)
#         print(candidates)
        cand_inds = [inds[c] for c in candidates]
#         print(cand_inds)
        cand_rows = df.loc[cand_inds].copy()
        cand_rows['distance'] = dists
        cand_rows = cand_rows[cand_rows['swapped'] == 0]
        if len(cand_rows) < k:
            num_to_query *= 2
            continue
        return cand_rows.head(k)
    cand_rows = df[(df['TOTAL'] == t) & (df['18_PLUS'] == a) & (df['swapped'] == 0) & (df['blockid'] != row['blockid'])].head(k).copy()
    cand_rows['distance'] = cand_rows.apply(block_distance, axis=1)
    return cand_rows

In [141]:
row = merged.iloc[0]
# print(row)
#print(merged.iloc[150860])
close = find_k_closest(row, merged, 5)
print(close)
# print(close[['TOTAL', '18_PLUS']])

[0.00212752 0.00532372 0.00628224 0.00628224 0.00628224]
[150860  35475 316278 316280 316279]
         YEAR    STATE  STATEA          COUNTY  COUNTYA  COUSUBA  TRACTA  \
539541   2010  Georgia      13  Houston County      153    92352   21400   
108226   2010  Georgia      13  Houston County      153    92352   21400   
1223370  2010  Georgia      13  Houston County      153    92352   21400   
1223372  2010  Georgia      13  Houston County      153    92352   21400   
1223371  2010  Georgia      13  Houston County      153    92352   21400   

         BLKGRPA  BLOCKA        NAME  ...  AGE_ACCURACY  frequency  U  prob  \
539541         1    1052  Block 1052  ...          True     552198  3   0.6   
108226         1    1035  Block 1035  ...          True     552198  4   1.0   
1223370        1    1040  Block 1040  ...          True     552198  2   0.3   
1223372        1    1040  Block 1040  ...          True     552198  2   0.3   
1223371        1    1040  Block 1040  ...          Tru

In [87]:
# Should try to make this a kdtree implementation
# Because there are so few (age, voting_age) pairs, maybe make a separate one for each?
# One challenge -- don't support removal. Will probably have to do a doubling search over k to get enough
def get_k_closest(row, df, k, UB=.01):
    matches = df[(df['swapped'] == 0) & (df['id'] != row['id']) & (df['18_PLUS'] == row['18_PLUS']) & (df['TOTAL'] == row['TOTAL'])].copy()
    matches['other_lat'] = row['INTPTLAT10']
    matches['other_lon'] = row['INTPTLON10']
    close_matches = matches[((matches['INTPTLAT10'] - matches['other_lat']).abs() < UB) & (matches['INTPTLON10'] - matches['other_lon']).abs()].copy()
    close_matches['distance'] = close_matches.apply(block_distance, axis=1)
    close_matches = close_matches.nsmallest(k, 'distance')
#     print(close_matches['distance'].max())
    if close_matches['distance'].max() < UB:
        return close_matches
#     print('failed')
    if UB < 1:
        return get_k_closest(row, df, k, UB*2)
    matches['distance'] = matches.apply(block_distance, axis=1)
    matches = matches.nsmallest(k, 'distance')
    return matches

In [45]:
merged = old_merged.copy()

In [160]:
%%time
hh_1s = []
hh_2s = []
dists = []
merged['swapped'] = 0
# cols = ['TOTAL', '18_PLUS', 'id']
# cols_to_swap = flagging + ['household.id', 'prob', 'U', 'frequency']
num_matches = 5
s = .0093
print('Total number of swaps', int(s*num_rows))
for i, row in merged.iterrows():
    if i % 1000 == 0:
        print(i)
    if merged['swapped'].sum() >= num_rows*s:
        break
    if merged.loc[i, 'swapped'] == 1:
        continue
    do_swap = random() < row['prob']
    if not do_swap:
        continue
    matches = find_k_closest(row, merged, num_matches)
    m = matches.sample()
    partner_index = m.index[0]
    m = m.reset_index().iloc[0]
    hh_1s.extend([row['household.id'], m['household.id']])
    hh_2s.extend([m['household.id'], row['household.id']])
    dists.extend([m['distance'], m['distance']])
    if i == partner_index:
        print(i, partner_index)
    if merged.loc[i, 'swapped'] == 1:
        print(i)
        print(merged.loc[i])
        print(row)
    if merged.loc[partner_index, 'swapped'] == 1:
        print(i, partner_index)
    assert i != partner_index
    assert merged.loc[i, 'swapped'] == 0
    assert merged.loc[partner_index, 'swapped'] == 0
    merged.loc[[i, partner_index], 'swapped'] = 1
partners = pd.DataFrame({'hh_1': hh_1s, 'hh_2': hh_2s, 'distance': dists})

Total number of swaps 33599
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
CPU times: user 1min 45s, sys: 5.19 s, total: 1min 50s
Wall time: 1min 55s


In [161]:
just_pairs = partners[['hh_1', 'hh_2']]
just_pairs.head()
print(len(just_pairs))
print(merged['swapped'].sum())
assert len(just_pairs) == merged['swapped'].sum()

33600
33600


In [162]:
swapped_df = merged.merge(
    just_pairs,
    left_on = 'household.id',
    right_on = 'hh_1',
    how = 'left',
    validate = 'one_to_one',
)
swapped_df.drop(columns=['hh_1', 'INTPTLAT10', 'INTPTLON10', 'COUNTY', 'NAME', 'COUSUBA', 'BLKGRPA', 'ACCURACY', 'AGE_ACCURACY'], inplace=True)
swapped_df.head()

,YEAR,STATE,STATEA,COUNTYA,TRACTA,BLOCKA,BLOCK_TOTAL,BLOCK_18_PLUS,TOTAL,W,...,NUM_HISP,18_PLUS,HH_NUM,frequency,U,prob,id,household.id,swapped,hh_2
0,2010,Georgia,13,153,21400,1034,1,1,1,0,...,1,1,0,102,4,1.0,021400-153-1034,021400-153-1034-0000,1,021400-153-1040-0001
1,2010,Georgia,13,299,950200,3025,1,1,1,0,...,1,1,0,102,4,1.0,950200-299-3025,950200-299-3025-0000,1,950200-299-3024-0000
2,2010,Georgia,13,23,790100,1102,1,1,1,0,...,0,1,0,268,4,1.0,790100-023-1102,790100-023-1102-0000,1,790100-023-1101-0001
3,2010,Georgia,13,35,150100,4082,1,1,1,0,...,0,1,0,268,4,1.0,150100-035-4082,150100-035-4082-0000,1,050102-207-2018-0001
4,2010,Georgia,13,63,40412,3031,1,1,1,0,...,0,1,0,268,4,1.0,040412-063-3031,040412-063-3031-0000,1,040412-063-3020-0003


In [163]:
swap_subset = swapped_df['swapped'] == 1
expanded = swapped_df.loc[swap_subset, 'hh_2'].str.split('-', expand=True)
print(expanded.head())
swapped_df.loc[swap_subset, 'COUNTYA'] = pd.to_numeric(expanded[1])
swapped_df.loc[swap_subset, 'TRACTA'] = pd.to_numeric(expanded[0])
swapped_df.loc[swap_subset, 'BLOCKA'] = pd.to_numeric(expanded[2])
swapped_df.loc[swap_subset, 'household.id'] = swapped_df.loc[swap_subset, 'hh_2']
swapped_df.rename({'id': 'blockid'}, inplace=True, axis=1)
del swapped_df['hh_2']
swapped_df.head()

        0    1     2     3
0  021400  153  1040  0001
1  950200  299  3024  0000
2  790100  023  1101  0001
3  050102  207  2018  0001
4  040412  063  3020  0003


,YEAR,STATE,STATEA,COUNTYA,TRACTA,BLOCKA,BLOCK_TOTAL,BLOCK_18_PLUS,TOTAL,W,...,TWO_OR_MORE,NUM_HISP,18_PLUS,HH_NUM,frequency,U,prob,blockid,household.id,swapped
0,2010,Georgia,13,153,21400,1040,1,1,1,0,...,0,1,1,0,102,4,1.0,021400-153-1034,021400-153-1040-0001,1
1,2010,Georgia,13,299,950200,3024,1,1,1,0,...,0,1,1,0,102,4,1.0,950200-299-3025,950200-299-3024-0000,1
2,2010,Georgia,13,23,790100,1101,1,1,1,0,...,0,0,1,0,268,4,1.0,790100-023-1102,790100-023-1101-0001,1
3,2010,Georgia,13,207,50102,2018,1,1,1,0,...,0,0,1,0,268,4,1.0,150100-035-4082,050102-207-2018-0001,1
4,2010,Georgia,13,63,40412,3020,1,1,1,0,...,0,0,1,0,268,4,1.0,040412-063-3031,040412-063-3020-0003,1


## Write to file 

In [166]:
# Make sure you want to do this. It may overwrite something
with open(get_swapped_file(), 'w') as f:
    swapped_df.to_csv(f)